In [1]:
import legume
import chickpea.fields as fields
import pickle
import numpy as np

import matplotlib.pyplot as plt

# Chickpea Fields Tutorial
The Chickpea.fields is module provides a class for manipulating and managing field profiles of solutions of
guided mode expansions generated by legume.

To start we must generate a set of solutions, in this case a L3 photonic crystal cavity.
We first generate the geometry and photonic crystal object.

In [2]:
Nx, Ny = 16, 16

# Regular PhC parameters
ra = 0.234
dslab = 0.4355
n_slab = 2.21

# Initialize a lattice and PhC
lattice = legume.Lattice([Nx, 0], [0, Ny*np.sqrt(3)/2])

# Make x and y positions in one quadrant of the supercell
# We only initialize one quadrant because we want to shift the holes symmetrically
xp, yp = [], []
nx, ny = Nx//2 + 1, Ny//2 + 1
for iy in range(ny):
    for ix in range(nx):
        xp.append(ix + (iy%2)*0.5)
        yp.append(iy*np.sqrt(3)/2)

# Move the first two holes to create the L4/3 defect
xp[0] = 2/5
xp[1] = 6/5
nc = len(xp)

# Initialize PhC
phc = legume.PhotCryst(lattice)

# Add a layer to the PhC
phc.add_layer(d=dslab, eps_b=n_slab**2)

# Apply holes symmetrically in the four quadrants
for ic, x in enumerate(xp):
    yc = yp[ic] if yp[ic] == 0 else yp[ic]
    xc = x if x == 0 else xp[ic]
    phc.add_shape(legume.Circle(x_cent=xc, y_cent=yc, r=ra))
    if nx-0.6 > xp[ic] > 0 and (ny-1.1)*np.sqrt(3)/2 > yp[ic] > 0:
        phc.add_shape(legume.Circle(x_cent=-xc, y_cent=-yc, r=ra))
    if nx-1.6 > xp[ic] > 0:
        phc.add_shape(legume.Circle(x_cent=-xc, y_cent=yc, r=ra))
    if (ny-1.1)*np.sqrt(3)/2 > yp[ic] > 0 and nx-1.1 > xp[ic]:
        phc.add_shape(legume.Circle(x_cent=xc, y_cent=-yc, r=ra))

We now generate the guided mode expansion solutions.

Setting Generate to True will regenerate the gme object with solutions. Otherwise, it will be loaded from the aux_files folder.

In [3]:
Generate = True

In [4]:
if not Generate:
    gme = pickle.load(open("./aux_files/Field_Tutorial_GME_Cav.p", 'rb'))
else:
    # Number of PhC periods in x and y directions

    gme = legume.GuidedModeExp(phc = phc)

    options = {'gmode_inds': [0],
               'verbose': True,
               'eig_solver': 'eigsh',
               'eig_sigma': 0.42,
               'numeig': 10,
               'gradients': 'approx',
               'compute_im': False}

    gme.run(**options)
    pickle.dump(gme, open("./aux_files/Field_Tutorial_GME_Cav.p", 'wb'))

Running k-point 1 of 1

KeyboardInterrupt: 

We will now generate the mode profiles of the modes we generated using the XYfield object.

Note that there are multiple ways of generating a XYfield. We will generate from a gme object
and make sure to specify the gme object in the constructor.

In [ ]:
res = 5*np.array([Nx, Ny])
field_1 = fields.XYField(res = res, # The resolution of the field profile (x,y)
                         z_dimension=dslab/2, # The location of the cross section in the slab.
                                              # Here we choose the center of the slab.
                         polarization='TE', # The polarization of the field. In the 'TE' case the constructor generates
                                            # the H_z and E_x E_y components setting other components to zero.

                         gme=gme, mind=0, kind=0)
#

With the fields object we can generate each field component.

In [ ]:
fig_hz = field_1.visualize_field(field='h', component='z', val='re')
fig_hz.axes[0].set_title('Re(H_z) field component.')


fig_ex = field_1.visualize_field(field='e', component='x', val='re')
fig_ex.axes[0].set_title('Re(E_x) field component.')

fig_ey = field_1.visualize_field(field='e', component='y', val='re')
fig_ey.axes[0].set_title('Re(E_y) field component.')

We can also generate the magnitude of the electric field using the 'normalize' flag.

In [ ]:
fig_exy = field_1.visualize_field(field='e', component='y', normalize=True)
fig_exy.axes[0].set_title('|E_xy| field component.')
#

The eps_dist function returns the permitivity distribution associated with the field.
It will match the resolution of the generated field. This is useful for mode volume calculations.

In [ ]:
eps = field_1.eps_dist()
plt.imshow(eps, cmap='Greys')


